<a href="https://colab.research.google.com/github/AayushSaini03/NIDS_project/blob/main/Darknet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report

df = pd.read_csv('/Darknet.CSV', sep=',', on_bad_lines='skip')

print(df.shape)
display(df.head())

(141530, 85)


,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Label.1
0,10.152.152.11-216.58.220.99-57158-443-6,10.152.152.11,57158,216.58.220.99,443,6,24/07/2015 04:09:48 PM,229,1,1,0,0,0,0,0.000000,0.000000,0,0,0.0000,0.000000,0.00000,8733.624454,229.000000,0.000000,229,229,0,0.000000,0.000000,0,0,0,0.000000,0.00000,0,0,0,0,0,0,20,20,4366.812227,4366.812227,0,0,0.000000,0.00000,0.000000,2,0,0,0,2,0,0,0,1,0.000000,0.000000,0.0000,0,0,0,0,0,0,0,0,0,0,1892,1047,0,20,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,Non-Tor,AUDIO-STREAMING
1,10.152.152.11-216.58.220.99-57159-443-6,10.152.152.11,57159,216.58.220.99,443,6,24/07/2015 04:09:48 PM,407,1,1,0,0,0,0,0.000000,0.000000,0,0,0.0000,0.000000,0.00000,4914.004914,407.000000,0.000000,407,407,0,0.000000,0.000000,0,0,0,0.000000,0.00000,0,0,0,0,0,0,20,20,2457.002457,2457.002457,0,0,0.000000,0.00000,0.000000,2,0,0,0,2,0,0,0,1,0.000000,0.000000,0.0000,0,0,0,0,0,0,0,0,0,0,1987,1047,0,20,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,Non-Tor,AUDIO-STREAMING
2,10.152.152.11-216.58.220.99-57160-443-6,10.152.152.11,57160,216.58.220.99,443,6,24/07/2015 04:09:48 PM,431,1,1,0,0,0,0,0.000000,0.000000,0,0,0.0000,0.000000,0.00000,4640.371230,431.000000,0.000000,431,431,0,0.000000,0.000000,0,0,0,0.000000,0.00000,0,0,0,0,0,0,20,20,2320.185615,2320.185615,0,0,0.000000,0.00000,0.000000,2,0,0,0,2,0,0,0,1,0.000000,0.000000,0.0000,0,0,0,0,0,0,0,0,0,0,2049,1047,0,20,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,Non-Tor,AUDIO-STREAMING
3,10.152.152.11-74.125.136.120-49134-443-6,10.152.152.11,49134,74.125.136.120,443,6,24/07/2015 04:09:48 PM,359,1,1,0,0,0,0,0.000000,0.000000,0,0,0.0000,0.000000,0.00000,5571.030641,359.000000,0.000000,359,359,0,0.000000,0.000000,0,0,0,0.000000,0.00000,0,0,0,0,0,0,20,20,2785.515320,2785.515320,0,0,0.000000,0.00000,0.000000,2,0,0,0,2,0,0,0,1,0.000000,0.000000,0.0000,0,0,0,0,0,0,0,0,0,0,2008,1047,0,20,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,Non-Tor,AUDIO-STREAMING
4,10.152.152.11-173.194.65.127-34697-19305-6,10.152.152.11,34697,173.194.65.127,19305,6,24/07/2015 04:09:45 PM,10778451,591,400,64530,6659,131,0,109.187817,22.283313,498,0,16.6475,46.833714,6604.75239,91.942711,10887.324242,11412.466412,78158,13,10778451,18268.561017,11786.143089,81171,126,10747836,26936.932331,15897.73845,78158,307,1,0,0,0,11820,8000,54.831627,37.111084,0,498,71.876008,56.93647,3241.761603,1,0,0,659,991,0,0,0,0,71.948537,109.187817,16.6475,0,0,0,0,659,6605,0,65,0,6,1382,2320,581,20,0,0,0,0,1.437765e+15,3.117718e+06,1.437765e+15,1.437765e+15,Non-Tor,AUDIO-STREAMING


In [ ]:
import numpy as np
drop_cols = ["Flow ID", "Src IP", "Dst IP", "Timestamp", "Label.1"]
df.drop(columns=drop_cols, errors="ignore", inplace=True)

# Drop rows with missing or infinite values
df.replace([np.inf, -np.inf], pd.NA, inplace=True)
df.dropna(inplace=True)

# Encode non-numeric features
non_numeric_cols = df.select_dtypes(include=['object']).columns.drop('Label')  # Exclude label
for col in non_numeric_cols:
    df[col] = LabelEncoder().fit_transform(df[col])

# Label encoding for multi-class target
le = LabelEncoder()
y = le.fit_transform(df["Label"])
print("Label mapping:", dict(zip(le.classes_, le.transform(le.classes_))))

# One-hot encode the label for multi-class classification
y_categorical = to_categorical(y)

# Prepare feature matrix
X = df.drop(columns=["Label"])
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_categorical, test_size=0.3, random_state=42)

# Reshape for LSTM [samples, time steps, features]
X_train_lstm = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

Label mapping: {'Non-Tor': np.int64(0), 'NonVPN': np.int64(1), 'Tor': np.int64(2), 'VPN': np.int64(3)}


In [ ]:
# Build LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(1, X_train.shape[1]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32))
model.add(Dropout(0.2))
model.add(Dense(y_categorical.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train model
model.fit(X_train_lstm, y_train, epochs=10, batch_size=64, validation_data=(X_test_lstm, y_test))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 1, 64)          │        36,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 49,412 (193.02 KB)

 Trainable params: 49,412 (193.02 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1548/1548 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.7628 - loss: 0.6940 - val_accuracy: 0.8780 - val_loss: 0.3285
Epoch 2/10
1548/1548 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.8792 - loss: 0.3287 - val_accuracy: 0.8972 - val_loss: 0.2594
Epoch 3/10
1548/1548 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.8954 - loss: 0.2719 - val_accuracy: 0.9051 - val_loss: 0.2412
Epoch 4/10
1548/1548 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - accuracy: 0.9036 - loss: 0.2518 - val_accuracy: 0.9143 - val_loss: 0.2236
Epoch 5/10
1548/1548 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.9106 - loss: 0.2344 - val_accuracy: 0.9223 - val_loss: 0.2164
Epoch 6/10
1548/1548 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.9161 - loss: 0.2260 - val_accuracy: 0.9197 - val_loss: 0.2056
Epoch 7/10
1548/1548 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.9196 - loss: 0.2145 - val_accuracy: 0.9245 - val_loss: 0.2042
Epoch 8/10
1548/1548 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.9204 - loss: 0.

In [ ]:
# Evaluate the model

import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# Predict class probabilities
y_pred = np.argmax(model.predict(X_test_lstm), axis=1)
# Convert one-hot encoded y_test back to class indices
y_true = np.argmax(y_test, axis=1)

# Evaluate
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=le.classes_))

1327/1327 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
[[27868    68     3    26]
 [  189  5318     3  1653]
 [    0    68   327    23]
 [  195   825     2  5877]]
              precision    recall  f1-score   support

     Non-Tor       0.99      1.00      0.99     27965
      NonVPN       0.85      0.74      0.79      7163
         Tor       0.98      0.78      0.87       418
         VPN       0.78      0.85      0.81      6899

    accuracy                           0.93     42445
   macro avg       0.90      0.84      0.87     42445
weighted avg       0.93      0.93      0.93     42445



In [ ]:
import plotly.graph_objects as go
import numpy as np
from sklearn.metrics import confusion_matrix

# Generate class labels and confusion matrix
class_labels = le.classes_  # ['Non-Tor', 'NonVPN', 'Tor', 'VPN']
cm = confusion_matrix(y_true, y_pred)

# Create heatmap
fig = go.Figure(data=go.Heatmap(
    z=cm,
    x=class_labels,
    y=class_labels,
    colorscale='Blues',
    hovertemplate='Predicted: %{x}<br>Actual: %{y}<br>Count: %{z}<extra></extra>',
    showscale=True
))

fig.update_layout(
    title='Confusion Matrix (Multi-Class LSTM)',
    xaxis_title='Predicted Label',
    yaxis_title='True Label',
    yaxis=dict(autorange='reversed')
)

fig.show()